In [1]:
import os
import argparse
import torchvision
from tqdm import tqdm
import pytorch_lightning
# import pytorchvideo.data
import torch.utils.data
from sklearn.metrics import accuracy_score,auc,roc_auc_score
from script.video_data_loader import VideoDataLoader
import pytorchvideo.models.resnet
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
# import math
from script.get_model import get_model

from script.transformation import * #ToTensor, CenterCrop_resize, RandomCrop, RandomRotate,RandomHorizontalFlip,RandomVerticalFlip

In [2]:
# parser = argparse.ArgumentParser(description='Model_training.')
# parser.add_argument("-r_dir", "--root_dir", help="Enter the path to root dir.", default=r"D:\data")
# parser.add_argument("-d_dir", "--data_dir", help="Enter the path to data dir data\train.", default="RGB_D\RGB")
# parser.add_argument("-batch_size", "--batch_size", help="Enter the batch_size.", default=32)
# parser.add_argument("-fps", "--frame_rate", help="Enter the desired frame rate.", default=30)
# parser.add_argument("-clip_d", "--clip_duration", help="Enter the desired frame rate.", default=16)
# parser.add_argument("-num_classes", "--num_classes", help="Enter Number of classes.", default=10)
# parser.add_argument("-model", "--model", help="Select from the list ['slow_r50' ,'c2d_r50' ,'r3d_18','slowfast_r50','r2plus1D_18']", default='r3d_18')
# parser.add_argument("-weights", "--pre_trained", help="Should model use pretrained weights are not True/False.", default='True')
# parser.add_argument("-m", "--mode", help="Data_type the microscope: RGB, Depth,RGB_D.", default="RGB")
# parser.add_argument("-lr", "--learning_rate", help="Learning rate screach  0.01, 0.001 transfer Learning", default=0.001)
# parser.add_argument("-ep", "--epochs", help="Learning rate screach  0.01, 0.001 transfer Learning", default=100)

# args = parser.parse_args()


In [3]:
batch_size = 4
num_frames = 8
num_epochs = 10
frame_rate = 10
image_size = 112
num_classes = 60
pre_trained= 'r2plus1D_18' #args.model
weight = True
lr = 0.001  #screach  0.01, 0.001 transfer Learning
# patience = 20
mode = 'RGB_D'
path_to_folder= r'D:\data'
data_folder = r'RGB_D\RGB'
# hello= transform=transforms.Compose([Rescale(256),RandomCrop(224),ToTensor()
transform_train=transforms.Compose([ToTensor(num_frames,frame_rate),
                                                    CenterCrop_resize(256),RandomCrop(224),RandomHorizontalFlip(0.5),
                                                    RandomRotate(30),RandomVerticalFlip(0.5)])

transform_test=transforms.Compose([ToTensor(num_frames,frame_rate),
                                                    CenterCrop_resize(256)])


In [4]:
train_dataset = VideoDataLoader(root_dir=path_to_folder,csv_dir=r'data_files\RGB_D_train_1.csv', 
                                transform=transform_train,frame_rate=frame_rate, image_size=image_size, num_frames = num_frames,
                                mode=mode)

test_dataset = VideoDataLoader(root_dir=path_to_folder,csv_dir=r'data_files\RGB_D_test_1.csv',
                               transform=transform_test,frame_rate=frame_rate, image_size=image_size, num_frames = num_frames,
                               mode=mode)


# print(len(train_dataset))

train = DataLoader(train_dataset, batch_size=batch_size,shuffle=True, num_workers=0,pin_memory=True)
test = DataLoader(test_dataset, batch_size=batch_size,shuffle=True, num_workers=0,pin_memory=True)


D:\data\RGB_D/RGB
D:\data\RGB_D/Depth
D:\data\RGB_D/RGB
D:\data\RGB_D/Depth


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [6]:
net= get_model(pretrained_model='r3d_18',num_classes=num_classes, weight=weight,mode='RGB_D')
net.to(device)

VideoResNet(
  (stem): BasicStem(
    (0): Conv3d(4, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (conv2): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1):

In [7]:
for data in train:
    print(data['videos'].shape)  
    break

torch.Size([4, 4, 8, 224, 224])


In [11]:
import os
if os.path.exists(r"C:\Users\wajah\Desktop\Training\model\r3d_18_self.pth"):
    print("Hello")

Hello


In [10]:
if os.path.exists(model_path):
    net = torch.load(model_path)
else:
    net = get_model(pretrained_model=pre_trained,num_classes=num_classes, weight=weight,mode=args.mode)

NameError: name 'model_path' is not defined

In [ ]:
# def make_kinetics_resnet():
#   return pytorchvideo.models.resnet.create_resnet(
#       input_channel=4, # RGB input from Kinetics
#       model_depth=101, # For the tutorial let's just use a 50 layer network
#       model_num_class=60, # Kinetics has 400 classes so we need out final head to align
#       norm=nn.BatchNorm3d,
#       activation=nn.ReLU,
#   )

# model = make_kinetics_resnet()
# print(model)

In [ ]:
import torch.optim as optim
import torch.nn as nn
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
scheduler = StepLR(optimizer, step_size=20, gamma=0.1)

In [ ]:

def train_loop(model, train_dataloader, device, optimizer, criterion):
    """
    One Iteration across the training set
    Args:
        :param model: solution.Basset()
        :param train_dataloader: torch.utils.data.DataLoader
                                 Where the dataset is solution.BassetDataset
        :param device: torch.device
        :param optimizer: torch.optim
        :param critereon: torch.nn (output of get_critereon)

    :Returns: output dict with keys: total_score, total_loss
    values of each should be floats
    (if you want to display losses and/or scores within the loop, you may print them to screen)

    Make sure your loop works with arbitrarily small dataset sizes!

    Note: you don’t need to compute the score after each training iteration.
    If you do this, your training loop will be really slow!
    You should instead compute it every 50 or so iterations and aggregate ...
    """

#     output = {'total_score': 0.,
#               'total_loss': 0.}
    model.train()
    LOSSES = 0
    COUNTER = 0
    ITERATIONS = 0
    p_labels = torch.tensor(()).to(device)
    y_labels = torch.tensor(()).to(device)
    
#     for i,data in enumerate(train_dataloader,0):
    with tqdm(train_dataloader, unit="batch") as tepoch:
        for data in tepoch:
            optimizer.zero_grad()
            
            inputs, y_true = data['videos'].to(device), data['labels'].to(device)
            outputs = net(inputs)
            _,y_pred = torch.max(outputs,1)
            loss = criterion(outputs, y_true.long())
            loss.backward()
            optimizer.step()
        
            n = y_true.size(0)  #total number training example in batch
            LOSSES += loss.sum().data.cpu() * n        
            y_labels = torch.cat((y_labels, y_true.long()), 0)
            p_labels = torch.cat((p_labels, y_pred), 0)
            tepoch.set_postfix(loss=loss.item())
            COUNTER += n
#             if COUNTER == 20:
#                 break

        
        y_labels = y_labels.cpu()
        p_labels = p_labels.cpu()
        p_labels= p_labels.detach().numpy()
        y_labels = y_labels.detach().numpy()
    
        x = accuracy_score(y_labels, p_labels) #roc_auc_score#
        
    

        
    return x, LOSSES / float(COUNTER)


In [ ]:
def valid_loop(model, valid_dataloader, device, optimizer, criterion):
    """
    One Iteration across the training set
    Args:
        :param model: solution.Basset()
        :param train_dataloader: torch.utils.data.DataLoader
                                 Where the dataset is solution.BassetDataset
        :param device: torch.device
        :param optimizer: torch.optim
        :param critereon: torch.nn (output of get_critereon)

    :Returns: output dict with keys: total_score, total_loss
    values of each should be floats
    (if you want to display losses and/or scores within the loop, you may print them to screen)

    Make sure your loop works with arbitrarily small dataset sizes!

    Note: you don’t need to compute the score after each training iteration.
    If you do this, your training loop will be really slow!
    You should instead compute it every 50 or so iterations and aggregate ...
    """


    model.eval()
    LOSSES = 0
    COUNTER = 0
    ITERATIONS = 0
    p_labels = torch.tensor(()).to(device)
    y_labels = torch.tensor(()).to(device)
    with torch.no_grad():
        with tqdm(valid_dataloader, unit="batch") as tepoch:
#         for i,data in enumerate(valid_dataloader,0):
            for data in tepoch:
                inputs, y_true = data['videos'].to(device), data['labels'].to(device)
                outputs = net(inputs.to(torch.float32))
                _,y_pred = torch.max(outputs,1)
                loss = criterion(outputs, y_true.long())
#                 writer.add_scalar("Loss/validation", loss, epoch)
            
                n = y_true.size(0)  #total number training example in batch
                LOSSES += loss.sum().data.cpu() * n        
                y_labels = torch.cat((y_labels, y_true.long()), 0)
                p_labels = torch.cat((p_labels, y_pred), 0)
                COUNTER += n
                
                if COUNTER==20:
                    break


    y_labels = y_labels.cpu()
    p_labels = p_labels.cpu()
    p_labels= p_labels.detach().numpy()
    y_labels = y_labels.detach().numpy()
    
    x = accuracy_score(y_labels, p_labels)
    return x, LOSSES / float(COUNTER)





In [ ]:
import numpy as np
v_loss= np.array(10.0)

train_losses, valid_losses = [], []
train_accuracy, valid_accuracy = [], []
for epoch in range(num_epochs):
    tqdm.write(f"====== Epoch {epoch} ======>")
    
    train_acc,train_loss = train_loop(net, train, device, optimizer, criterion)
    valid_acc,valid_loss = valid_loop(net, train, device, optimizer, criterion)
    train_losses.append(train_loss)
    train_accuracy.append(train_acc)
    valid_losses.append(valid_loss)
    valid_accuracy.append(valid_acc)
    if v_loss>valid_loss.cpu().numpy():
        v_loss=valid_loss.cpu().numpy()
        torch.save(net,'r3d_18_self.pth')
        print('validation loss improved and new model is saved')
    
    
    print('validation_loss',valid_loss)
    curr_lr = optimizer.param_groups[0]['lr']
    
    print(f'Epoch {epoch}\ \
        Training Loss: {train_loss}\ \
        Training Accuracy: {train_acc}\ \
        Validation Loss:{valid_loss}\ \
        Validation Loss:{valid_acc}\ \
        LR:{curr_lr}')

    scheduler.step()

print(f"===== Best validation loss: {min(valid_losses):.3f} =====>")
print(f"===== Best validation Accuracy: {max(valid_accuracy):.3f} =====>")
writer.flush()    
#     break

In [ ]:
# print(f"===== Best validation loss: {min(valid_losses):.3f} =====>")
# print(f"===== Best validation Accuracy: {max(train_accuracy):.3f} =====>")
writer.close()
# !pip install tensorboard
# import tensorboard
# tensorboard --logdir=runs

In [ ]:
i = 0

for param in net.parameters():
#     if i < 45:
    param.requires_grad = True
#         print(name)
#     i += 1
    

In [ ]:
import numpy as np
# v_loss= np.array(10.0)
# train_losses, valid_losses = [], []
# train_accuracy, valid_accuracy = [], []
for epoch in range(num_epochs):
    tqdm.write(f"====== Epoch {epoch} ======>")
    
    train_acc,train_loss = train_loop(net, train_datagen, device, optimizer, criterion)
    valid_acc,valid_loss = valid_loop(net, test_datagen, device, optimizer, criterion)
    
    train_losses.append(train_loss)
    train_accuracy.append(train_acc)
    valid_losses.append(valid_loss)
    valid_accuracy.append(valid_acc)
    if v_loss>valid_loss.cpu().numpy():
        v_loss=valid_loss.cpu().numpy()
        torch.save(net,'r3d_18_aug.pth')
        print('validation loss improved and new model is saved')
    
    
    print('validation_loss',valid_loss)
    curr_lr = optimizer.param_groups[0]['lr']
    
    print(f'Epoch {epoch}\ \
        Training Loss: {train_loss}\ \
        Training Accuracy: {train_acc}\ \
        Validation Loss:{valid_loss}\ \
        Validation Loss:{valid_acc}\ \
        LR:{curr_lr}')

    scheduler.step(valid_loss)

print(f"===== Best validation loss: {min(valid_losses):.3f} =====>")
print(f"===== Best validation Accuracy: {max(valid_accuracy):.3f} =====>")
    
#     break

In [ ]:
# print(valid_acc, valid_loss)

torch.save(net,'r3d_18.pth')
import numpy as np
loss= []
np.save('r3d_18.npy',np.array(loss.append([train_losses, train_accuracy,valid_losses, valid_accuracy])))

In [ ]:
print(f"===== Best validation loss: {min(train_losses):.3f} =====>")
print(f"===== Best validation Accuracy: {max(train_accuracy):.3f} =====>")

In [ ]:
loss = np.load('test.npy')

In [ ]:
import matplotlib.pyplot as plt
lw = 2
alpha=0.6
epochs = range(0,num_epochs)

fig, axs = plt.subplots(1, 2, num=None, figsize=(14, 6), dpi=200, facecolor='w', edgecolor='k')
ax = axs[1]


ax.plot(epochs, train_losses, 'green', label='Training Loss', linestyle='solid', linewidth=lw)
ax.plot(epochs, valid_losses, 'green', label='Validation Loss',linestyle='dashed', linewidth=lw)

ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
#     plt.ylim([0,20])
ax.grid(True)
ax.set_title(f"Training and Validation Loss")
# ax.legend(handlelength=2)
ax.legend(loc='best', bbox_to_anchor=(0., -0.22, 1., .102),
           ncol=2, mode="expand", fontsize='small', borderaxespad=0.)



ax = axs[0]


ax.plot(epochs, train_accuracy, 'red', label='Training Loss', linestyle='solid', linewidth=lw)
ax.plot(epochs, valid_accuracy, 'red', label='Validation Loss',linestyle='dashed', linewidth=lw)

ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
#     plt.ylim([0,20])
ax.grid(True)
ax.set_title(f"Training and Validation Accuracy:  {valid_acc:.3f}%")
# ax.legend(handlelength=2)
ax.legend(loc='best', bbox_to_anchor=(0., -0.22, 1., .102),
           ncol=2, mode="expand", fontsize='small', borderaxespad=0.)



In [ ]:
torch.save(net, r'C:\Users\wajah\Desktop\pytorch_model\model\r3d_18.pth') 

In [ ]:
torch.save(net,'r3d_18.pth')

In [ ]:
print(v_loss)

In [ ]:
print(valid_loss.cpu().numpy())

In [ ]:
import random
for x in range(100):
    print(random.random())